In [2]:
import pandas as pd
from itertools import permutations, combinations, product

In [66]:
year = 2022

Bring in our predictions

In [3]:
df_pred = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness/data/2022/predictions/phase2_submissions_xg.csv')
df_pred.head()

,ID,Pred
0,2022_1103_1104,0.159308
1,2022_1103_1112,0.064552
2,2022_1103_1116,0.182135
3,2022_1103_1120,0.061031
4,2022_1103_1124,0.092754


Excellent, now we have our submission file into a dataframe with ID and Predictions. From here we'll need to bring in the first round teams and see who we predict will win.

In [4]:
#read in the team data
df_teams = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness2023/data/raw/MTeams.csv')
df_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023


In [5]:
#read in the tournament slot data
df_slots = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness2023/data/raw/MNCAATourneySlots.csv')
df_slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [67]:
df_slots_thisyear = df_slots[df_slots['Season']==year]
df_slots_thisyear = df_slots_thisyear.reset_index(drop=True)
df_slots_thisyear.head(48)

,Season,Slot,StrongSeed,WeakSeed
0,2022,R1W1,W01,W16
1,2022,R1W2,W02,W15
2,2022,R1W3,W03,W14
3,2022,R1W4,W04,W13
4,2022,R1W5,W05,W12
5,2022,R1W6,W06,W11
6,2022,R1W7,W07,W10
7,2022,R1W8,W08,W09
8,2022,R1X1,X01,X16
9,2022,R1X2,X02,X15


In [54]:
#read in the tournament seed data data
df_seeds = pd.read_csv('/Users/tamraheberling/Documents/data_ml/MarchMadness2023/data/raw/MNCAATourneySeeds.csv')
df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [68]:
df_seeds_thisyear = df_seeds[df_seeds['Season']==year]
df_seeds_thisyear = df_seeds_thisyear.reset_index(drop=True)
df_seeds_thisyear.head(20)

,Season,Seed,TeamID
0,2022,W01,1124
1,2022,W02,1246
2,2022,W03,1345
3,2022,W04,1417
4,2022,W05,1388
5,2022,W06,1400
6,2022,W07,1293
7,2022,W08,1314
8,2022,W09,1266
9,2022,W10,1362


Eliminate the letters for the playin games

In [115]:
df_seeds_thisyear.Seed.replace(['W12a','Y16a','X11a','Z16a'],['W12','Y16','X11','Z16'],inplace=True)
vals_to_drop=['W12b','X11b','Y16b','Z16b']
df_seeds_thisyear = df_seeds_thisyear[df_seeds_thisyear.Seed.isin(vals_to_drop) == False]
df_seeds_thisyear.index = range(64)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [116]:
print(df_seeds_thisyear[0:32])

    Season Seed  TeamID
0     2022  W01    1124
1     2022  W02    1246
2     2022  W03    1345
3     2022  W04    1417
4     2022  W05    1388
5     2022  W06    1400
6     2022  W07    1293
7     2022  W08    1314
8     2022  W09    1266
9     2022  W10    1362
10    2022  W11    1439
11    2022  W12    1231
12    2022  W13    1103
13    2022  W14    1463
14    2022  W15    1389
15    2022  W16    1313
16    2022  X01    1211
17    2022  X02    1181
18    2022  X03    1403
19    2022  X04    1116
20    2022  X05    1163
21    2022  X06    1104
22    2022  X07    1277
23    2022  X08    1129
24    2022  X09    1272
25    2022  X10    1172
26    2022  X11    1323
27    2022  X12    1308
28    2022  X13    1436
29    2022  X14    1286
30    2022  X15    1168
31    2022  X16    1209


# Helpful Definitions

In [100]:
'''Make a disctionary of the rank of each team and the probability that they win the round'''
def prob_win_rank(rank1, rank2, dict_region, dict_prob):
    teamid1 = dict_region[rank1]
    teamid2 = dict_region[rank2]
    prob_teamid1, prob_teamid2 = prob_win(teamid1,teamid2)
    dict_prob.update({rank1: prob_teamid1,rank2: prob_teamid2})
    return dict_prob

In [121]:
'''Given two teamIDs, calculate the probabality that each team wins the matchup'''
def prob_win(teamid1=int,teamid2=int):
    if teamid1 < teamid2:
        id_pred = str(year)+'_'+str(teamid1)+'_'+str(teamid2)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        prob_teamid1 = pred
        prob_teamid2 = 1-pred
    elif teamid2 < teamid1:
        id_pred = str(year)+'_'+str(teamid2)+'_'+str(teamid1)
        pred = float(df_pred.Pred[df_pred.ID==id_pred])
        prob_teamid2 = pred
        prob_teamid1 = 1-pred
    return prob_teamid1, prob_teamid2

In [132]:
'make a dictionary of the team seed and their id for each region'
def seed_id(int1=int, int2=int):
    seeds = [i+1 for i in range(16)]
    TeamIDs = [int(df_seeds_thisyear.TeamID[j]) for j in range(int1,int2)]
    dict_out = dict(zip(seeds,TeamIDs))
    return dict_out

In [96]:
'given a tuple, give the list containing the list of matchups in the next round'
def tup_to_matchups(tup=tuple, half_len_of_tuple=int):
    matchup_list = [[tup[i],tup[-i-1]] for i in range(half_len_of_tuple)]
    return matchup_list

In [221]:
'''function to calculate all the possible round results and their probabilities'''
def round_results(round_num=int, half_of_teams_in_round=int, tup_match=tuple, prob=float, round_results_dict=dict):
    #calculate all possible combinations of ranks in tup_match as a list of tuples
    round_matchups = tup_to_matchups(tup_match,half_of_teams_in_round)
    possible_combos = list(product(*round_matchups))
    '''use dict to pull the probability that each team in the tuple wins and multiply together to 
    calculate the probability of that tuple'''
    for i in possible_combos:
        #start with probability 1
        prob_combo = 1
        #calculate the probablity of each matchup and multiply together to get the probablity of the tuple emmerging from the round
        for j in i:
            prob_j = round1_prob_by_rank[j]
            prob_combo = prob_combo*prob_j
        #multiply with the probability of tup_match from the previous round(s) to get the probability for that version of the bracket
        if round_num==1:
            prob_start = 1
        else:
            prob_start = prob
        prob_combo = prob_start*prob_combo
        nest_tup = tup_match, i
        dict_temp = {nest_tup:prob_combo}
        round_results_dict.update(dict_temp)
    return(round_results_dict)

# Round: 1
For each match up in each region, we'll calculate the possible outcome tuples and the probabilities of each tuple occuring. Because the number of possible brackets is enormous, we'll eliminate the tuples that are highly unlikely in order to reduce the computation needs moving forward. 

In [133]:
'Calculate the rank and team id for each region'
W_region = seed_id(0,16)
X_region = seed_id(16,32)
Y_region = seed_id(32,48)
Z_region = seed_id(48,64)
X_region

{1: 1211,
 2: 1181,
 3: 1403,
 4: 1116,
 5: 1163,
 6: 1104,
 7: 1277,
 8: 1129,
 9: 1272,
 10: 1172,
 11: 1323,
 12: 1308,
 13: 1436,
 14: 1286,
 15: 1168,
 16: 1209}

In [125]:
'create matchups for round1, same for all regions'
tup_round1 = tuple(i+1 for i in range(16))
matchup_list = tup_to_matchups(tup_round1, 8)
print(matchup_list)

[[1, 16], [2, 15], [3, 14], [4, 13], [5, 12], [6, 11], [7, 10], [8, 9]]


In [138]:
'''caluclate the probability that each team wins the round for each region, 
the rank and probability are returned in a dictionary for each region'''
dict_prob_W = {}
dict_prob_X = {}
dict_prob_Y = {}
dict_prob_Z = {}
for match in matchup_list:
    dict_prob_W = prob_win_rank(match[0],match[1],W_region,dict_prob_W)
    dict_prob_X = prob_win_rank(match[0],match[1],X_region,dict_prob_X)
    dict_prob_Y = prob_win_rank(match[0],match[1],Y_region,dict_prob_Y)
    dict_prob_Z = prob_win_rank(match[0],match[1],Z_region,dict_prob_Z)
print(dict_prob_Z)

{1: 0.88333344, 16: 0.11666655999999997, 2: 0.88901017, 15: 0.11098983, 3: 0.88451424, 14: 0.11548576, 4: 0.8340561200000001, 13: 0.16594388, 5: 0.77801055, 12: 0.22198945000000003, 6: 0.5855874, 11: 0.4144126, 7: 0.5572577000000001, 10: 0.4427423, 8: 0.4962958, 9: 0.5037042}


In [184]:
'''each possible outcome from round1 and the probability it occurs for reach region, 
returned as a dictionary with a tuple of 8 teams emmerging from round1 and the probability of it happening'''
round1_results_pred_W = round1_results(dict_prob_W)
round1_results_pred_X = round1_results(dict_prob_X)
round1_results_pred_Y = round1_results(dict_prob_Y)
round1_results_pred_Z = round1_results(dict_prob_Z)
round1_results_pred_W

{(1, 2, 3, 4, 5, 6, 7, 8): 0.006626408219937722,
 (1, 2, 3, 4, 5, 6, 7, 9): 0.006532857091775057,
 (1, 2, 3, 4, 5, 6, 10, 8): 0.006004155457810215,
 (1, 2, 3, 4, 5, 6, 10, 9): 0.005919389246900944,
 (1, 2, 3, 4, 5, 11, 7, 8): 0.003535289450654986,
 (1, 2, 3, 4, 5, 11, 7, 9): 0.0034853785025948845,
 (1, 2, 3, 4, 5, 11, 10, 8): 0.0032033081490848003,
 (1, 2, 3, 4, 5, 11, 10, 9): 0.00315808408783577,
 (1, 2, 3, 4, 12, 6, 7, 8): 0.0059303141511228155,
 (1, 2, 3, 4, 12, 6, 7, 9): 0.00584659042617522,
 (1, 2, 3, 4, 12, 6, 10, 8): 0.005373428091837036,
 (1, 2, 3, 4, 12, 6, 10, 9): 0.005297566441994848,
 (1, 2, 3, 4, 12, 11, 7, 8): 0.0031639126902042133,
 (1, 2, 3, 4, 12, 11, 7, 9): 0.0031192448110526996,
 (1, 2, 3, 4, 12, 11, 10, 8): 0.0028668055176207007,
 (1, 2, 3, 4, 12, 11, 10, 9): 0.002826332174974918,
 (1, 2, 3, 13, 5, 6, 7, 8): 0.0020506418064916983,
 (1, 2, 3, 13, 5, 6, 7, 9): 0.0020216910011553154,
 (1, 2, 3, 13, 5, 6, 10, 8): 0.0018580763191460372,
 (1, 2, 3, 13, 5, 6, 10, 9): 0.001

In [143]:
'only keep tuples with a probability of occuring greater than 0.001'
round1_W = {key:val for key, val in round1_results_pred_W.items() if val >= 0.001}
round1_X = {key:val for key, val in round1_results_pred_X.items() if val >= 0.001}
round1_Y = {key:val for key, val in round1_results_pred_Y.items() if val >= 0.001}
round1_Z = {key:val for key, val in round1_results_pred_Z.items() if val >= 0.001}

# Round 2
For each tuple of teams in the round1 dictionaries, calculate the matchups, the probability of each team winning the second round, possible tuples to emmerge from round2, and probabilities of it happening

In [223]:
round2_W = {}
for key, val in round1_W.items():
    tup_seeds = key
    prob_tup = val
    round2_W = round_results(2, 4, tup_seeds, prob_tup, round2_W)
print(round2_W)
print(len(round2_W))

{((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 3, 4)): 0.04144555412877649, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 3, 5)): 0.02731702858728388, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 6, 4)): 0.03451735548649301, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 2, 6, 5)): 0.022750608754131412, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 3, 4)): 0.024406584898570366, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 3, 5)): 0.01608653548992614, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 6, 4)): 0.020326686055098356, ((1, 2, 3, 4, 5, 6, 7, 8), (1, 7, 6, 5)): 0.013397448187725761, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 3, 4)): 0.020539474707120257, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 3, 5)): 0.013537698542981499, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 6, 4)): 0.01710601691483359, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 2, 6, 5)): 0.011274684653151373, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 3, 4)): 0.012095348795525153, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 3, 5)): 0.007972121395552065, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 6, 4)): 0.010073443651182562, ((1, 2, 3, 4, 5, 6, 7, 8), (8, 7, 6, 5)): 0.

In [ ]:
round2_W = {key:val for key, val in round1_results_pred_W.items() if val >= 0.001}